# Векторы в R (часть 2)

## Правила переписывания (recycling)

Ранее уже приводился пример работы векторной арифметики на двух векторах разной длины. Приведём несколько общих правил для таких случаев:
1. Длина вектора-результата равна длине большего из векторов
2. Меньший вектор дублируется (переписывается), пока его длина не совпадёт с длиной большего вектора
3. Если длина большего вектора не делится нацело на длину меньшего, то выдаётся предупреждение

In [4]:
1:5 + 1:2
# (1, 2, 3, 4, 5) + (1, 2, 1, 2, 1) = (2, 4, 4, 6, 6)

Warning message in 1:5 + 1:2:
"длина большего объекта не является произведением длины меньшего объекта"

[1] 2 4 4 6 6

Данная особенность оказывается очень удобной при необходимости производить массовые операции над векторами:

In [5]:
1:9 * 4

[1]  4  8 12 16 20 24 28 32 36

In [6]:
(1:6) ^ 3

[1]   1   8  27  64 125 216

In [10]:
seq(0.1, 2.5, by = 0.2) > 2

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
[13]  TRUE

## Доступ к элементам вектора

Чтение и запись элементов осуществляется с помощью оператора **[ ]**:

In [14]:
x <- seq(10, 50, by = 5)
ind <- 3:5
x[ind] # Обращение к подмножеству элементов вектора по индексу ind
arr <- x[ind] # Запись подмножества элементов вектора x по индексу ind в переменную arr
arr <- arr / 5
x[ind] <- arr # Замена подмножества элементов вектора x по индексу ind на множество значений вектора arr
x

[1] 20 25 30

[1] 10 15  4  5  6 35 40 45 50

Для индекса ind существует несколько правил:
1. Отсутствие индекса

In [15]:
x[]
# Обращение ко всему вектору

[1] 10 15  4  5  6 35 40 45 50

2. Положительные индексы (элементы по указанному индексу)

In [17]:
x[1] # Конкретный элемент
x[2:5] # Подмножество
x[c(1, 2, 5:7, x[5])] # Несколько подмножеств

[1] 10

[1] 15  4  5  6

[1] 10 15  6 35 40 35

3. Отрицательные индексы (элементы кроме указанных по индексу)

In [20]:
x[-2] # Кроме 2-го элемента
x[-(3:5)] # Кроме элементов с 3-го по 5-й
x[-length(x)] # Все элементы, кроме последнего

[1] 10  4  5  6 35 40 45 50

[1] 10 15 35 40 45 50

[1] 10 15  4  5  6 35 40 45

4. Логические индексы (элементы, соответствующие значению *TRUE*)

In [23]:
x[rep(c(TRUE, FALSE), 5)] # Элементы на нечётных позициях - TRUE, на чётных - FALSE
x[1:9 %% 2 != 0] # Идентичный пример

[1] 10  4  6 40 50

[1] 10  4  6 40 50

5. Именные индексы (элементы по указанным именам)

In [8]:
y <- c(six = 6, four = 4, two = 2, zero = 0)
y[c("two", "six", "Hello world")]
# При неизвестном имени имеем NA
names(y) <- NULL

two  six <NA> 
   2    6   NA

## *all* и *any*

С помощью функции ***all*** можно проверить, удовлетворяют ли *все* элементы вектора некторому условию. ***any*** проверяет соответствие *некоторых* элементов условию:

In [5]:
all(y < 7)
all(y > 0)

[1] TRUE

[1] FALSE

In [6]:
any(y > 0)
any(y < 0)

[1] TRUE

[1] FALSE

## *which*

Функция ***which*** возвращает индексы элементов, удовлетворяющих условию:

In [9]:
which(y > 2)

[1] 1 2

***which.min*** и ***which.max*** возвращают позицию наименьшего и наибольшего элементов соответственно:

In [10]:
which.min(y)
which.max(y)

[1] 4

[1] 1

## Атрибуты объектов

- "Cвойства" объектов, по типу *length*, существуют всегда, вне зависимости от нашего желания
- *Атрибуты* (names, dim...) могут присутствовать при необходимости (можно задавать функцией ***attr***)

In [1]:
z <- c(1, 2, 3)
names(z) <- c("one", "two", "three")
attr(z, "author") <- "Translator"
attributes(z) # Возвращает атрибуты объекта

$names
[1] "one"   "two"   "three"

$author
[1] "Translator"

In [3]:
attributes(z) <- NULL
attributes(z)
# Удаление атрибутов

NULL

## Интересная задача на векторизацию

Пусть есть вектор чисел от 1 до 50. Необходимо все числа, делящиеся на 2, возвести в квадрат, делящиеся на 3, - в куб, а делящиеся и на 2, и на 3, - в шестую степень. Приведём банальный метод решения, который доступен в любом языке программирования:

In [38]:
a <- 1:20
for (i in a) {
    if (!i %% 6) {
        a[i] <- a[i]^6
    } else if (!i %% 3) {
        a[i] <- a[i]^3
    } else if (!i %% 2) {
        a[i] <- a[i]^2
    }
}
a

# В данном решении задачи цикл for проходит по всем элементам вектора и проверяет условия делимости

[1]        1        4       27       16        5    46656        7       64
 [9]      729      100       11  2985984       13      196     3375      256
[17]       17 34012224       19      400

Приведём пример более красивого решения, использующего векторизацию:

In [39]:
a <- 1:20
#Узнать, какие элементы делятся на 2 (или на любое другое число), легко можно с помощью векторизации
!a %% 2 # ! - логическое отрицание
# a %% 2 == 0 - идентично

[1] FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE
[13] FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE

In [40]:
# За счёт логической индексации мы можем произвести нужные нам операции только над элементами со значением TRUE
a[!a %% 2] <- a[!a %% 2]^2 
a

# При возведении в степень количество различных простых множителей никак не меняется, поэтому проверять делимость можем
# по самому вектору a

[1]   1   4   3  16   5  36   7  64   9 100  11 144  13 196  15 256  17 324  19
[20] 400

In [41]:
# Теперь для тройки (6 = 2 * 3 - правила возведения в степень)
a[!a %% 3] <- a[!a %% 3]^3 
a

[1]        1        4       27       16        5    46656        7       64
 [9]      729      100       11  2985984       13      196     3375      256
[17]       17 34012224       19      400

Вместо громоздкого решения, использующего цикл, мы решили задачу буквально в две строки:  
`a[!a %% 2] <- a[!a %% 2]^2`  
`a[!a %% 3] <- a[!a %% 3]^3`

## Еще одна задача

В векторе случайных чисел найти 2 соседних с максимальной разницей.  
Не будем рассматривать решения через цикл, они довольно очевидны. Приведём векторизированное решение:

In [45]:
set.seed(22)
# Не стоит уделять много внимания этой функции, она лишь ограничивает "случайность" чисел (они каждый раз одинаковые)
x <- sample(1:20, 10)
# Создали вектор случайных чисел от 1 до 20

[1]  6  9 10 12 17 11  1  5 15  3

In [46]:
x[-1] # Вектор x без первого элемента

[1]  9 10 12 17 11  1  5 15  3

In [47]:
x[-length(x)] # Вектор x без последнего элемента

[1]  6  9 10 12 17 11  1  5 15

In [48]:
x[-1] - x[-length(x)] # Все разницы между соседними элементами

[1]   3   1   2   5  -6 -10   4  10 -12

In [50]:
m <- which.max(abs(x[-1] - x[-length(x)])) # Максимум из взятых по модулю разниц
x[c(m, m + 1)] # Выводим два искомых элемента

[1] 15  3